In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
sys.path.insert(0, "/home/ylu/project/spateo-release")
import spateo as st
from spateo.alignment.methods.morpho_class import *
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-10 23:02:01.410382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 23:02:01.568713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-10 23:02:01.568739: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-10 23:02:02.258095: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "/home/ylu/project/MOSTA/data/"
results_folder = "./results/Spateo/"
figures_folder = "./results/figures/Spateo"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(1, 14)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"E16.5_E2S{i}.MOSTA.h5ad")))

100%|█| 13/13 [01:01<00:00,  4.70s


In [4]:
spatial_key = "spatial"

In [5]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [6]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [7]:
## Perform the Spateo alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    top_genes_num = 5000
    top_genes_1 = np.argsort(-slice1.X.sum(0)).A[0][:top_genes_num] if slice1.shape[1] > top_genes_num else np.arange(slice1.shape[1])
    top_genes_2 = np.argsort(-slice2.X.sum(0)).A[0][:top_genes_num] if slice2.shape[1] > top_genes_num else np.arange(slice2.shape[1])
    slice1 = slice1[:,top_genes_1]
    slice2 = slice2[:,top_genes_2]
    time_start = time.time()
    adata = ad.concat([slice1,slice2], label='batch')
    PCA(adata, 'X', 'X_pca', 50, 'cuda')
    slice1.obsm['X_pca'] = adata[adata.obs['batch'] == '0'].obsm['X_pca'].copy()
    slice2.obsm['X_pca'] = adata[adata.obs['batch'] == '1'].obsm['X_pca'].copy()
    align_slices = [slice1, slice2]
    morpho_model = Morpho_pairwise(
        sampleA=align_slices[1],
        sampleB=align_slices[0],
        spatial_key=spatial_key,
        key_added=key_added,
        
        # iter_key_added='align_spatial_iter',
        partial_robust_level=10,
        # nn_init=False,
        lambdaVF=1e1,
        beta=1e-1,
        sigma2_init_scale=1,
        device='1',
        sparse_calculation_mode=False,
        use_chunk=False,
        chunk_capacity=2,
        # rep_layer=['X'],
        # rep_field=['layer'],
        # dissimilarity=['kl'],
        rep_layer=['X_pca'],
        rep_field=['obsm'],
        dissimilarity=['cos'],
        probability_type=['gauss'],
        nn_init=True,
        nn_init_weight=1,
        max_iter=1000,
        batch_size=4000,
    )
    P = morpho_model.run()
    align_slices[1].obsm[key_added] = morpho_model.optimal_RnA.copy()
    align_slices[0].obsm[key_added] = align_slices[0].obsm[spatial_key].copy()
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    pi = P.T
    inlier = np.where(pi.sum(axis=1) > 0.1)[0]
    matches = [inlier, np.argmax(pi[inlier,:], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|      | 0/12 [00:00<?, ?it/s]/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/merge.py:1291: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning:

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

/tmp/ipykernel_810160/627414376.py:17: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_810160/627414376.py:18: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----> Filtered all samples for common genes. There are 4668 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([124.8757, 124.8757])...
|-----------> Scale: tensor([[ 152.1436, -222.8499],
        [-130.4205,  294.9133]], device='cuda:0')...
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [29.8450s]
|-----> Key Parameters: gamma: 0.9880659580230713; sigma2: 0.0015743013937026262; probability_parameters: [tensor(0.0144, device='cuda:0')]


  8%| | 1/12 [00:58<10:47, 58.86s//home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/merge.py:1291: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning:

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

/tmp/ipykernel_810160/627414376.py:17: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_810160/627414376.py:18: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----> Filtered all samples for common genes. There are 4649 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([134.7789, 134.7789])...
|-----------> Scale: tensor([[ 117.2904, -496.9182],
        [ 152.0397, -223.2934]], device='cuda:0')...
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [32.1677s]
|-----> Key Parameters: gamma: 0.9900000095367432; sigma2: 0.0010000000474974513; probability_parameters: [tensor(0.0181, device='cuda:0')]


 17%|▏| 2/12 [01:58<09:52, 59.27s//home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/merge.py:1291: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning:

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

/tmp/ipykernel_810160/627414376.py:17: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_810160/627414376.py:18: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



...

 83%|▊| 10/12 [08:33<01:33, 46.95s/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/merge.py:1291: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning:

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

/tmp/ipykernel_810160/627414376.py:17: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_810160/627414376.py:18: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----> Filtered all samples for common genes. There are 4810 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([147.7430, 147.7430])...
|-----------> Scale: tensor([[-334.2280,  374.1857],
        [ 378.3135, -473.5271]], device='cuda:0')...
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [19.2840s]
|-----> Key Parameters: gamma: 0.9800925850868225; sigma2: 0.0028742754366248846; probability_parameters: [tensor(0.0078, device='cuda:0')]


 92%|▉| 11/12 [09:16<00:45, 45.67s/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/merge.py:1291: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning:

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

/tmp/ipykernel_810160/627414376.py:17: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_810160/627414376.py:18: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----> Filtered all samples for common genes. There are 4785 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([141.4339, 141.4339])...
|-----------> Scale: tensor([[-119.9698,  264.8565],
        [-335.0218,  375.0026]], device='cuda:0')...
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [19.5518s]
|-----> Key Parameters: gamma: 0.9834792017936707; sigma2: 0.003821425838395953; probability_parameters: [tensor(0.0088, device='cuda:0')]


100%|█| 12/12 [10:00<00:00, 50.04s


In [10]:
## Perform the Spateo alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis, _  = st.align.morpho_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="0",
        mode="SN-S",
        max_iter=200,
        verbose=True,
        partial_robust_level=1,
        beta=1e-1,
        lambdaVF=1e0,
        SVI_mode=True,
        nn_init=True,
        iter_key_added=None,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                     | 0/12 [00:00<?, ?it/s]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 27069 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 124.58662414550781.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [4.8952s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.42358991503715515; sigma2: 0.0015428022015839815
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [5.5501s]


  8%|███████████▊                                                                                                                                 | 1/12 [00:40<07:24, 40.45s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 27243 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 134.6942138671875.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [2.7581s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.4082432687282562; sigma2: 0.002117339987307787
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [4.4160s]


 17%|███████████████████████▌                                                                                                                     | 2/12 [01:21<06:48, 40.88s/it]

...

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                       | 10/12 [10:18<02:04, 62.40s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 27033 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 148.02999877929688.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [2.7562s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.4684436023235321; sigma2: 0.005542703904211521
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [4.3773s]


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎           | 11/12 [11:23<01:03, 63.08s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%|-----> Filtered all samples for common genes. There are 26908 common genes.
|-----> Coordinates normalization params:
|-----------> Scale: 141.57791137695312.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start morpho alignment] in progress: 100.0000%
|-----> [Start morpho alignment] finished [2.8033s]
|-----> Key Parameters: gamma: 0.9900000095367432; beta2: 0.42932796478271484; sigma2: 0.004915144760161638
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [4.3537s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [12:28<00:00, 62.35s/it]
